### Install arcos4py from pypi

In [ ]:
! pip install arcos4py

### Import packages

In [ ]:
import pandas as pd
from arcos4py import ARCOS
TAB20 = [
    "#1f77b4",
    "#aec7e8",
    "#ff7f0e",
    "#ffbb78",
    "#2ca02c",
    "#98df8a",
    "#d62728",
    "#ff9896",
    "#9467bd",
    "#c5b0d5",
    "#8c564b",
    "#c49c94",
    "#e377c2",
    "#f7b6d2",
    "#7f7f7f",
    "#c7c7c7",
    "#bcbd22",
    "#dbdb8d",
    "#17becf",
    "#9edae5",
]

### Example Dataset

The synthetic dataset contains 81 objects spaced on a 2D 9x9 lattice. Each object has an ID (column id) and can assume values 0 and 1 (column m), which correspond to an inactive and active state. The evolution of active states takes place over 8 consecutive time points (column t). Each cell moves slightly over time, hence a light wiggle around the initial position.

dataset location: arcos4py/sample_data/arcos_data.csv"

For an in-depth explanation of the parameters check out the documentation

### Set variables for loading and processing of the data

For an in-depth explanation of the parameters check out the documentation

In [ ]:
# path to data
DATA: str = "sample_data/arcos_data.csv"

# column names in DATA
POSITION_COLUMN_NAMES: list = ['x', 'y']
FRAME_COLUMN_NAME: str = "t"
OBJECT_ID_COLUMN_NAME: str = 'id'
MEASUREMENT_COLUMN_NAME: str = "m"

# column name with collective event track ids
COLLID_COLUMN_NAME: str = "collid"

# values for clipping and interpolation
INTERPOLATE: bool = False
CLIP: bool = False
CLIP_LOW: float = 0.001
CLIP_HIGH: float = 0.999

# binarization parameters
DETRENDING_METHOD: str = "none"   # can be "none", "runmed" or "lm"
SMOOTHK: int = 3
BIASK: int = 25                     # needed only if BINARIZATION_METHOD method = runmed
PEAK_THRESHOLD: float = 0.5         # needed only if BINARIZATION_METHOD method = runmed
DEGREE_OF_POLYNOMIAL: int = 1       # needed only if BINARIZATION_METHOD method = lm
BINARIZATION_THRESHOLD: float = 0.5

# event detection parameters
EPS: float = 2
MIN_CLUSTER_SIZE: int = 2
NPREV_FRAMES: int = 1

# filter parameters
MIN_COLLECTIVE_EVENT_DURATION: int = 2
MIN_TOTAL_EVENT_SIZE: int = 5

### Load dataset

In [ ]:
df = pd.read_csv(DATA)

# print first rows of data
df.head()

### Detect collective events

In [ ]:
# create ARCOS object
ts = ARCOS(data=df, posCols=POSITION_COLUMN_NAMES, frame_column=FRAME_COLUMN_NAME, id_column=OBJECT_ID_COLUMN_NAME, measurement_column=MEASUREMENT_COLUMN_NAME, clid_column=COLLID_COLUMN_NAME)

# interpolate and clip if necessary
if INTERPOLATE:
    ts.interpolate_measurements() # interpolation of missing values, optional
if CLIP:
    ts.clip_meas(CLIP_LOW, CLIP_HIGH) # clipping of measurement value, optional

# detrend and binarize data
ts.bin_measurements(smoothK=SMOOTHK, biasK=BIASK, peakThr=PEAK_THRESHOLD, binThr=BINARIZATION_THRESHOLD, polyDeg=DEGREE_OF_POLYNOMIAL, biasMet=DETRENDING_METHOD) # detrending and binarization

# detect and track collective events
collev = ts.trackCollev(eps=EPS, minClsz=MIN_CLUSTER_SIZE, nPrev=NPREV_FRAMES)

# print first rows of detected collective events
collev.head()

## filter collective events

In [ ]:
from arcos4py.tools import filterCollev

In [ ]:
filterer = filterCollev(data = collev, frame_column= FRAME_COLUMN_NAME, collid_column=COLLID_COLUMN_NAME, obj_id_column=OBJECT_ID_COLUMN_NAME) # initialize class
collev_filtered = filterer.filter(coll_duration=MIN_COLLECTIVE_EVENT_DURATION, coll_total_size=MIN_TOTAL_EVENT_SIZE) #
collev_filtered.head()

### Collective event Stats

In [ ]:
from arcos4py.tools import calcCollevStats

In [ ]:
stats = calcCollevStats().calculate(collev_filtered, FRAME_COLUMN_NAME, COLLID_COLUMN_NAME, OBJECT_ID_COLUMN_NAME, POSITION_COLUMN_NAMES)
stats.head()

### Plotting Module

In [ ]:
from arcos4py import plotting

In [ ]:
ndl = plotting.NoodlePlot(collev_filtered,COLLID_COLUMN_NAME, OBJECT_ID_COLUMN_NAME, FRAME_COLUMN_NAME, POSITION_COLUMN_NAMES[0], POSITION_COLUMN_NAMES[1])
ndl_plot = ndl.plot(POSITION_COLUMN_NAMES[0])

In [ ]:
data_plots = plotting.dataPlots(df,FRAME_COLUMN_NAME, MEASUREMENT_COLUMN_NAME, OBJECT_ID_COLUMN_NAME)

In [ ]:
hist = data_plots.histogram()

In [ ]:
dens = data_plots.density_plot()

In [ ]:
xt_plot = data_plots.position_t_plot({POSITION_COLUMN_NAMES[0]}, n=50)

In [ ]:
detrend_plot = plotting.plotOriginalDetrended(collev,FRAME_COLUMN_NAME, MEASUREMENT_COLUMN_NAME,f'{MEASUREMENT_COLUMN_NAME}.resc',OBJECT_ID_COLUMN_NAME).plot_detrended()

In [ ]:
coll_dur_size_scatter = plotting.statsPlots(stats).plot_events_duration('total_size','duration')